In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from finta import TA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import KFold

In [2]:
# Load data
GM_path = Path("../files/GM.csv")
VIX_path = Path("../files/^VIX.csv")
FEDFUNDS_path = Path("../files/FEDFUNDS (1).csv")
SPY_index = Path("../files/SPY.csv")

In [3]:
gm_df = pd.read_csv(GM_path, index_col="Date")
fear_index_df = pd.read_csv(VIX_path, index_col="Date").rename(columns={"Close": "Fear_index"})
spy_index_df = pd.read_csv(SPY_index, index_col="Date").rename(columns={"Close": "SPY_index"})
fedfunds_df = pd.read_csv(FEDFUNDS_path, index_col="DATE")

In [4]:
# Convert index to datetime index
fedfunds_df.index = pd.to_datetime(fedfunds_df.index)

# Resample the fed_funds_df to have daily frequency and forward fill the values
fedfunds_df_monthly = fedfunds_df.resample('D').ffill()


In [5]:
# Concatenate dataframes
concatenated_df = pd.concat([gm_df, fear_index_df['Fear_index'], spy_index_df['SPY_index']], axis=1)
concatenated_df.index = pd.to_datetime(concatenated_df.index)


In [6]:
# Merge with fedfunds_df
concatenated_df = pd.merge(concatenated_df, fedfunds_df_monthly, left_index=True, right_index=True)


In [7]:
# Drop rows with NaN values
concatenated_df = concatenated_df.dropna()


In [8]:
# Shift target variable
concatenated_df['Target'] = concatenated_df['Close'].shift(-5)
concatenated_df = concatenated_df.dropna()

In [9]:
concatenated_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target
2014-04-30,33.840000,34.540001,33.840000,34.480000,26.403223,14037800.0,13.41,188.309998,0.09,35.070000
2014-05-01,34.669998,35.400002,34.540001,34.900002,26.724840,19572600.0,13.25,188.330002,0.09,34.849998
2014-05-02,35.029999,35.330002,34.860001,34.970001,26.778439,11396700.0,12.91,188.059998,0.09,34.230000
2014-05-05,34.840000,34.889999,34.630001,34.750000,26.609976,8449100.0,13.29,188.419998,0.09,34.820000
2014-05-06,35.070000,35.139999,34.750000,34.750000,26.609976,10114500.0,13.80,186.779999,0.09,35.150002
2014-05-07,34.959999,35.110001,34.610001,35.070000,26.855013,9269700.0,13.40,187.880005,0.09,34.939999
2014-05-08,34.980000,35.430000,34.730000,34.849998,26.686544,10251500.0,13.43,187.679993,0.09,34.360001
2014-05-09,34.799999,34.900002,34.150002,34.230000,26.211781,16803500.0,12.92,187.960007,0.09,34.000000
2014-05-12,34.380001,34.939999,34.380001,34.820000,26.663578,9498900.0,12.23,189.789993,0.09,34.250000
2014-05-13,34.840000,35.189999,34.599998,35.150002,26.916273,9048300.0,12.13,189.960007,0.09,33.070000


In [10]:
# Calculate technical indicators using finta
data = concatenated_df.copy()  # Use the existing DataFrame concatenated_df
data['MA'] = TA.SMA(data, 20)  # 20-period Simple Moving Average
data['RSI'] = TA.RSI(data, 14)  # 14-period RSI

# Calculate Bollinger Bands correctly
bb_bands = TA.BBANDS(data, 20, 2)

# Assign Bollinger Bands values to DataFrame columns
data['BB_UPPER'] = bb_bands['BB_UPPER']
data['BB_MIDDLE'] = bb_bands['BB_MIDDLE']
data['BB_LOWER'] = bb_bands['BB_LOWER']

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Display the calculated technical indicators
data.tail()


,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2024-03-18,40.889999,40.919998,40.330002,40.820000,40.820000,16126600.0,14.33,512.859985,5.33,43.549999,40.0255,61.193289,41.374301,40.0255,38.676699
2024-03-19,40.919998,41.680000,40.720001,41.509998,41.509998,15936400.0,13.82,515.710022,5.33,44.000000,40.1505,65.123702,41.564825,40.1505,38.736175
2024-03-20,41.419998,42.889999,41.360001,42.849998,42.849998,16909300.0,13.04,520.479980,5.33,44.590000,40.3185,71.219972,42.141629,40.3185,38.495371
2024-03-21,42.919998,43.590000,42.840000,43.419998,43.419998,15268300.0,12.92,522.200012,5.33,45.349998,40.5225,73.353638,42.752324,40.5225,38.292675
2024-03-22,43.299999,43.660000,43.000000,43.060001,43.060001,9387400.0,13.06,521.210022,5.33,45.400002,40.6940,69.832357,43.150856,40.6940,38.237144


In [11]:
# Define features and target
X = concatenated_df.drop("Close", axis=1)
y = concatenated_df["Close"]

In [12]:
data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)


In [13]:
# Display the modified DataFrame
data.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2014-04-30,14037800.0,13.41,188.309998,0.09,35.070000,NaN,NaN,NaN,NaN,NaN
2014-05-01,19572600.0,13.25,188.330002,0.09,34.849998,NaN,100.000000,NaN,NaN,NaN
2014-05-02,11396700.0,12.91,188.059998,0.09,34.230000,NaN,100.000000,NaN,NaN,NaN
2014-05-05,8449100.0,13.29,188.419998,0.09,34.820000,NaN,66.004355,NaN,NaN,NaN
2014-05-06,10114500.0,13.80,186.779999,0.09,35.150002,NaN,66.004355,NaN,NaN,NaN


In [14]:
data_clean = data.dropna()
data_clean.index.rename('date', inplace=True)
data_clean.to_csv('../clean_data/GM_prepared_data.csv', index=True)

In [15]:
data_clean.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,,,
2014-05-28,12575800.0,11.68,191.380005,0.09,36.520000,34.3815,54.072814,35.609923,34.3815,33.153077
2014-05-29,10880900.0,11.57,192.369995,0.09,36.270000,34.3800,52.166905,35.607990,34.3800,33.152010
2014-05-30,14172100.0,11.40,192.679993,0.09,36.549999,34.3640,53.795503,35.571633,34.3640,33.156368
2014-06-02,11556900.0,11.58,192.899994,0.10,36.500000,34.3585,57.177346,35.555467,34.3585,33.161534
2014-06-03,17795900.0,11.87,192.800003,0.10,36.400002,34.3840,61.511328,35.636524,34.3840,33.131475


In [16]:
# Define date cutoff for data split
date_cutoff = "2022-04-30"

# Split data
X_train = X[X.index <= date_cutoff]
X_test = X[X.index > date_cutoff]
y_train = y[y.index <= date_cutoff]
y_test = y[y.index > date_cutoff]

In [17]:
# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# Define the number of folds
k = 10

# Initialize lists to store R-squared scores
train_r2_scores = []
test_r2_scores = []

# Initialize KFold
kf = KFold(n_splits=k, shuffle=True)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Example: Train with a smaller learning rate
from keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Compile the model
model.compile(optimizer=adam, loss='mean_squared_error')

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    
    # Evaluate the model on training data
    train_predictions = model.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_r2_scores.append(train_r2)
    
    # Evaluate the model on test data
    test_predictions = model.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_r2_scores.append(test_r2)

# Calculate average R-squared scores
avg_train_r2 = np.mean(train_r2_scores)
avg_test_r2 = np.mean(test_r2_scores)

print("Average R-squared (Train):", avg_train_r2)
print("Average R-squared (Test):", avg_test_r2)

/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 301us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step
Average R-squared (Train): -18.876992141338715
Average R-squared (Test): -19.136521130746814


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists to store metrics for selected models
selected_train_r2 = []
selected_test_r2 = []
selected_train_mae = []
selected_train_mse = []
selected_test_mae = []
selected_test_mse = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    # Define a new model for each fold
    model_fold = Sequential([
        Dense(32, activation='relu', input_shape=(X_train_cv.shape[1],)),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(16, activation='relu'),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(1)  # Output layer
    ])
    
    # Compile the model with Adam optimizer and mean squared error loss
    model_fold.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Train the model
    model_fold.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    
    # Evaluate the model on training data
    train_predictions = model_fold.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_mae = mean_absolute_error(y_train_cv, train_predictions)
    train_mse = mean_squared_error(y_train_cv, train_predictions)
    
    # Evaluate the model on test data
    test_predictions = model_fold.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_mae = mean_absolute_error(y_test_cv, test_predictions)
    test_mse = mean_squared_error(y_test_cv, test_predictions)
    
    # Append metrics to the lists
    selected_train_r2.append(train_r2)
    selected_test_r2.append(test_r2)
    selected_train_mae.append(train_mae)
    selected_train_mse.append(train_mse)
    selected_test_mae.append(test_mae)
    selected_test_mse.append(test_mse)

# Print metrics for selected models
for idx, (train_r2, test_r2, train_mae, train_mse, test_mae, test_mse) in enumerate(zip(selected_train_r2, selected_test_r2, selected_train_mae, selected_train_mse, selected_test_mae, selected_test_mse), start=1):
    print(f"Model {idx} - Train R-squared: {train_r2}, Test R-squared: {test_r2}, Train MAE: {train_mae}, Train MSE: {train_mse}, Test MAE: {test_mae}, Test MSE: {test_mse}")


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step
Model 1 - Train R-squared: 0.7828356894266528, Test R-squared: 0.795920727662786, Train MAE: 3.793532455804558, Train MSE: 16.174667588469536, Test MAE: 3.7862703503309185, Test MSE: 16.101839125695378
Model 2 - Train R-squared: 0.6632575427010032, Test R-squared: 0.5521617367880697, Train MAE: 4.937233427508605, Train MSE: 25.969070673122374, Test MAE: 4.831810280585261, Test MSE: 24.671791652665664
Model 3 - Train R-squared: 0.7302807123751969, Test R-squared: 0.7048504622811493, Train MAE: 4.2454347489691315, Train MSE: 20.3670955565614, Test MAE: 4.273050650498191, Test MSE: 20.52341163588271
Model 4 - Train R-squared: 0.6664298972709151, Test R-squared: 0.7059228903707351, Train MAE: 4.664604711607743, Train MSE: 24.241746621091792, Test MAE: 4.921049986114804, Test MSE: 27.60779798690174
Model 5 - Train R-squared: 0.7454666985832612, Test R-squared: 0.7680312729922737, Train MAE: 4.005560319864191, Tr

In [20]:
# Filter models where both Train R-squared and Test R-squared are less than 0.96
filtered_indices = [i for i, (train_r2, test_r2) in enumerate(zip(selected_train_r2, selected_test_r2)) if train_r2 < 0.96 and test_r2 < 0.96]

# Calculate the absolute difference between train R-squared and test R-squared values for filtered models
abs_diff_r2_filtered = np.abs(np.array(selected_train_r2)[filtered_indices] - np.array(selected_test_r2)[filtered_indices])

# Find the index of the model with the smallest absolute difference among filtered models
best_model_index = filtered_indices[np.argmin(abs_diff_r2_filtered)]

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.7236878634142989, Test R-squared: 0.7309712418054446, Train MAE: 4.3541661739567195, Train MSE: 20.76207968661613, Test MAE: 4.30328245369041, Test MSE: 19.65938574455488


In [21]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=0)

# Predict sequentially on each data point
all_predictions = model.predict(X_scaled)

# Ensure the number of predictions matches the original dataset
assert len(all_predictions) == len(X_scaled)

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame({'Actual': y, 'Predicted': all_predictions.flatten()}, index=X.index)

# Ensure index uniqueness in both the original dataset and predictions DataFrame
data_clean_unique_index = data_clean.index.drop_duplicates()
predictions_df = predictions_df.loc[data_clean_unique_index]

# Display the DataFrame
predictions_df


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step


,Actual,Predicted
date,,
2014-05-28,34.590000,24.052505
2014-05-29,34.450001,24.241785
2014-05-30,34.580002,24.018957
2014-06-02,34.860001,24.322723
2014-06-03,35.259998,24.293758
...,...,...
2024-03-18,40.820000,28.530863
2024-03-19,41.509998,28.850000
2024-03-20,42.849998,29.411497


In [22]:
predictions_df.to_csv('../predicted_data/GM_predicted_data.csv', index=True)

In [23]:
# Check for NaN values in the predictions DataFrame
nan_values = predictions_df.isnull().sum().sum()

if nan_values == 0:
    print("No NaN values found in the predictions DataFrame.")
    print(predictions_df)
else:
    print(f"Found {nan_values} NaN values in the predictions DataFrame. Please check your data or model.")


No NaN values found in the predictions DataFrame.
               Actual  Predicted
date                            
2014-05-28  34.590000  24.052505
2014-05-29  34.450001  24.241785
2014-05-30  34.580002  24.018957
2014-06-02  34.860001  24.322723
2014-06-03  35.259998  24.293758
...               ...        ...
2024-03-18  40.820000  28.530863
2024-03-19  41.509998  28.850000
2024-03-20  42.849998  29.411497
2024-03-21  43.419998  30.049303
2024-03-22  43.060001  30.140919

[2473 rows x 2 columns]


In [24]:
# Calculate the percentage difference between actual and predicted values
predictions_df['Percentage Difference (%)'] = ((predictions_df['Predicted'] - predictions_df['Actual']) / predictions_df['Actual']) * 100

# Display the DataFrame with percentage difference
predictions_df


,Actual,Predicted,Percentage Difference (%)
date,,,
2014-05-28,34.590000,24.052505,-30.463991
2014-05-29,34.450001,24.241785,-29.631976
2014-05-30,34.580002,24.018957,-30.540903
2014-06-02,34.860001,24.322723,-30.227416
2014-06-03,35.259998,24.293758,-31.101078
...,...,...,...
2024-03-18,40.820000,28.530863,-30.105677
2024-03-19,41.509998,28.850000,-30.498671
2024-03-20,42.849998,29.411497,-31.361730


In [25]:
# Calculate the absolute percentage difference for each data point
predictions_df['Abs_Percentage_Diff'] = abs((predictions_df['Actual'] - predictions_df['Predicted']) / predictions_df['Actual']) * 100

# Calculate the average percentage difference
avg_percentage_diff = predictions_df['Abs_Percentage_Diff'].mean()

print("Average Percentage Difference (%):", avg_percentage_diff)


Average Percentage Difference (%): 29.61182762481877
